In [6]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain.chat_models import ChatOpenAI

In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
KEY = os.getenv("OPENAI_API_KEY")

In [11]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-4o",temperature=0.5)

/var/folders/8j/ythr_y152qx4ycdfpz80jjf80000gn/T/ipykernel_15602/1807527743.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-4o",temperature=0.5)


In [13]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [15]:
RESPONSE_JSON = {
    "1": {
        "MCQ" : "Mulitple Choice Question",
        "Options" : {
            "A" : "First Choice",
            "B" : "Second Choice",
            "C" : "Third Choice",
            "D" : "Fourth Choice"
        }, 
        "Correct Answer" : "Correct Answer"
    }
}

In [16]:
TEMPLATE = """ 
Text: {text}
You are an expert MCQ Generator. Given the above text, your task is to \
generate a quiz of {number} multiple choice questions in {tone} tone.
These mulitple choice questions will be used by the professor to test the students so generate difficult questions.
You have to make sure that the questions are not repeated and then check if all the questions align with the given text or not.
You also have to make sure that you format your response as per the RESPONSE_JSON below and use it as a guide to generate the questions.

### RESPONSE_JSON
{response_json}
"""


In [17]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [18]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "Quiz", verbose = True)

/var/folders/8j/ythr_y152qx4ycdfpz80jjf80000gn/T/ipykernel_15602/2586721845.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "Quiz", verbose = True)


In [19]:
TEMPLATE2 = """"
You are expert english evaluator. You will check the quiz questions for grammar and spelling mistakes. 
You will be given with mulitple choice questions for {subject} students.
Your task is to evaluate the complexity of the questions and give a complete analysis of the quiz quiz. Only use a maximum of 50 words for the analysis. 
If the quiz is not as per the cognitive and analytical abilty of the students, then we must generate new questions such that it perfectly fits the 
student's abilities.

Quiz_MCQ:
{quiz}
"""